## Overview

## Movie Genre Analysis for Microsoft Movie Studio.

This project aims to to provide actionable insights for Microsoft's new movie studio. The goal is to identify commercially successful film types to guide genre selection decisions.

## Business Understanding

Microsoft, has decided to venture into the film industry by establishing a new movie studio. However, lack of experience in movie production necessitates a data-driven approach to navigate this new territory.This project aims to solve the problem of identifying commercially successful movie genres for Microsoft's new studio. Key objectives include:

1.Explore and analyze movie datasets to understand trends and patterns in the film industry.
2.Identify the types of movies that are performing well at the box office according to multiple criteria such as revenue, budget, box office perfomance, ratings, and genre.
3.Translate the findings into concrete business recommendations for Microsoft's new movie studio.


Primary Stakeholder of this project is Microsoft's new movie studio. The project will directly benefit the studio by providing:

1.Actionable recommendations on genres with high domestic gross.
2.Data-driven insights on audience reception verses gross return.
3.A competitive edge by identifying potentially lucrative niche genres.

# Data Understanding

To gain a comprehensive understanding of movie genre performance, we've utilized a combination of data sources;

1.tn.movie_budgets.csv: This table contains essential information about each movie, including its title, genre(s), release date, and production budget. This data allows us to categorize movies by genre and analyze box office performance within those categories.
2.tmdb.movies.csv: This table captures user ratings for various movies. While not a direct measure of box office success, user ratings can offer valuable insights into audience reception and potential genre preferences.
3.bom.movie_gross.csv: This dataset, contains details about a movie's domestic and international box office gross.

## Python Libraliries

In [1]:
# Importing relevant python libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import sqlite3

# Read the first CSV file bom.movies into a pandas DataFrame using bom_movies as our variable

In [ ]:
#reading the bom.movies using the file path
bom_movies = pd.read_csv(r"C:\Users\user\Desktop\DS_projects\phase_1_project_data\bom.movie_gross.csv")
# Display the first 5 rows
df.head()